In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from squadds.core.utils import set_huggingface_api_key

set_huggingface_api_key()

API key already exists in .env file.


In [3]:
from datasets import get_dataset_config_names
from datasets import load_dataset
from squadds import SQuADDS_DB
from squadds.interpolations.physics import ScalingInterpolator
from squadds import Analyzer
from squadds import AnsysSimulator
import numpy as np

db = SQuADDS_DB()
configs = get_dataset_config_names("SQuADDS/SQuADDS_DB")

## Select system style (qubit + RR)

In [4]:
db.select_system(['qubit','cavity_claw'])
db.select_qubit('TransmonCross')
db.select_cavity_claw('RouteMeander')
db.select_coupler('CLT')
df = db.create_system_df()

analyzer = Analyzer(db)
analyzer.target_param_keys()
target_params = {
                    "qubit_frequency_GHz": 5.523,
                    "cavity_frequency_GHz": 7.815,
                    "kappa_kHz": 400,
                    "resonator_type":"quarter",
                    "anharmonicity_MHz": -200,
                    "g_MHz": 30}

results = analyzer.find_closest(target_params=target_params,
                                        num_top=3,
                                        metric="Euclidean",
                                        display=True)
interpolator = ScalingInterpolator(analyzer, target_params)
design_df = interpolator.get_design()
best_device = results.iloc[0]
simulator = AnsysSimulator(analyzer, best_device)
design_options = best_device['design_options']


 g:\Shared drives\CavityCooling\DARPA\Simulations\SQUADDS\SQuADDS\squadds\core\analysis.py: 274
 g:\Shared drives\CavityCooling\DARPA\Simulations\SQUADDS\SQuADDS\squadds\core\analysis.py: 274
 g:\Shared drives\CavityCooling\DARPA\Simulations\SQUADDS\SQuADDS\squadds\core\analysis.py: 274


Kappa scaling: 1.0331772769572103
g scaling: 0.9903907788621945
alpha scaling: 0.9941204623259026
selected system: ['qubit', 'cavity_claw']


In [ ]:
design_options_dict_list = []
simulated_params_list = []   
sweep_list = list(np.arange(0,20.1,10))
design_options['qubit_options']['connection_pads']['readout']['ground_spacing'] = sweep_list
# design_options['ground_spacing'] = sweep_list
# design_df
# ansys_results = simulator.simulate(best_device)
sweep_results = simulator.sweep_qubit_cavity(best_device)
# design_options = ansys_results['design']['design_options']
# sim_results = ansys_results['sim_results']
# design_options_dict_list.append(design_options)
# simulated_params_list.append(sim_results)


In [5]:
best_device['design_options_qubit']

{'aedt_hfss_capacitance': 0,
 'aedt_hfss_inductance': 9.686e-09,
 'aedt_q3d_capacitance': 0,
 'aedt_q3d_inductance': 1e-08,
 'chip': 'main',
 'connection_pads': {'readout': {'claw_cpw_length': '40um',
   'claw_cpw_width': '10um',
   'claw_gap': '5.1um',
   'claw_length': '70um',
   'claw_width': '15um',
   'connector_location': '90',
   'connector_type': '0',
   'ground_spacing': '10um'}},
 'cross_gap': '30um',
 'cross_length': '230um',
 'cross_width': '30um',
 'gds_cell_name': 'my_other_junction',
 'hfss_capacitance': 0,
 'hfss_inductance': 9.686e-09,
 'hfss_mesh_kw_jj': 7e-06,
 'hfss_resistance': 0,
 'layer': '1',
 'orientation': '-90',
 'pos_x': '-1500um',
 'pos_y': '1200um',
 'q3d_capacitance': 0,
 'q3d_inductance': '10nH',
 'q3d_mesh_kw_jj': 7e-06,
 'q3d_resistance': 0}

In [ ]:
type(best_device['design_options'])

In [ ]:
best_device['design_options']['cavity_claw_options']

In [ ]:
best_device['design_options']['qubit_options']['connection_pads']['readout']['ground_spacing'] = sweep_list